In [ ]:
# Tutorial by www.pylessons.com
# Tutorial written for - Tensorflow 2.3.1

import os
import random
import gym
import pylab
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, Dense, Lambda, Add, Conv2D, Flatten, LSTM, Reshape
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras import backend as K
import cv2
import threading
from threading import Thread, Lock
import time
import tensorflow_probability as tfp
from typing import Any, List, Sequence, Tuple
import deepmind_lab

gpus = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_virtual_device_configuration(gpus[0],
            [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=4000)])
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
#os.environ['CUDA_VISIBLE_DEVICES'] = '-1' # -1:cpu, 0:first gpu


tfd = tfp.distributions


class ActorCritic(tf.keras.Model):
    def __init__(self, input_shape, action_space):
        super(ActorCritic, self).__init__()
        
        self.conv_1 = Conv2D(8, 3, 2, padding="valid", activation="relu")
        self.conv_2 = Conv2D(16, 3, 2, padding="valid", activation="relu")
        
        self.lstm = LSTM(64, name="lstm", return_sequences=True, return_state=True)
        
        self.flatten = Flatten()
        self.reshape = Reshape((15*15,16))
        self.dense_0 = Dense(256, activation='relu', name="dense_0")
        self.dense_1 = Dense(action_space, name="dense_1")
        self.dense_2 = Dense(1, name="dense_2")
        
    def call(self, X_input, memory_state, carry_state, training):
        batch_size = tf.shape(X_input)[0]
        
        conv_1 = self.conv_1(X_input)
        conv_2 = self.conv_2(conv_1)
        #print("conv_2.shape:" , conv_2.shape)
        
        conv_2_reshaped = self.reshape(conv_2)
        
        initial_state = (memory_state, carry_state)
        lstm_output, next_memory_state, next_carry_state = self.lstm(conv_2_reshaped, initial_state=initial_state, 
                                                                     training=training)
        
        X_input = self.flatten(lstm_output)
        X_input = self.dense_0(X_input)

        action_logit = self.dense_1(X_input)
        value = self.dense_2(X_input)
        
        return action_logit, value, next_memory_state, next_carry_state


def safe_log(x):
  """Computes a safe logarithm which returns 0 if x is zero."""
  return tf.where(
      tf.math.equal(x, 0),
      tf.zeros_like(x),
      tf.math.log(tf.math.maximum(1e-12, x)))


def take_vector_elements(vectors, indices):
    """
    For a batch of vectors, take a single vector component
    out of each vector.
    Args:
      vectors: a [batch x dims] Tensor.
      indices: an int32 Tensor with `batch` entries.
    Returns:
      A Tensor with `batch` entries, one for each vector.
    """
    return tf.gather_nd(vectors, tf.stack([tf.range(tf.shape(vectors)[0]), indices], axis=1))


huber_loss = tf.keras.losses.Huber(reduction=tf.keras.losses.Reduction.SUM)
sparse_ce = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction=tf.keras.losses.Reduction.SUM)
mse_loss = tf.keras.losses.MeanSquaredError()

workspace_path = '/home/kimbring2/stabilizing_transformers'
writer = tf.summary.create_file_writer(workspace_path + "/tensorboard/pong")

def _action(*entries):
  return np.array(entries, dtype=np.intc)

ACTIONS = {
      'look_left': _action(-20, 0, 0, 0, 0, 0, 0),
      'look_right': _action(20, 0, 0, 0, 0, 0, 0),
      'look_up': _action(0, 10, 0, 0, 0, 0, 0),
      'look_down': _action(0, -10, 0, 0, 0, 0, 0),
      'strafe_left': _action(0, 0, -1, 0, 0, 0, 0),
      'strafe_right': _action(0, 0, 1, 0, 0, 0, 0),
      'forward': _action(0, 0, 0, 1, 0, 0, 0),
      'backward': _action(0, 0, 0, -1, 0, 0, 0),
      'fire': _action(0, 0, 0, 0, 1, 0, 0),
      'jump': _action(0, 0, 0, 0, 0, 1, 0),
      'crouch': _action(0, 0, 0, 0, 0, 0, 1)
  }

ACTION_LIST = list(ACTIONS)

class PPOAgent:
    # PPO Main Optimization Algorithm
    def __init__(self, env_name):
        # Initialization
        # Environment and PPO parameters
        self.env_name = env_name       
        self.env = deepmind_lab.Lab(self.env_name, ['RGB_INTERLEAVED'],
                       {'fps': '15', 'width': '64', 'height': '64'})
        #env.reset(seed=1)
        
        self.action_size = len(ACTION_LIST)
        self.EPISODES, self.episode, self.max_average = 20000, 0, -21.0 # specific for pong
        self.lock = Lock()
        self.learning_rate = 0.0001

        self.ROWS = 80
        self.COLS = 60
        self.REM_STEP = 3

        # Instantiate plot memory
        self.scores, self.episodes, self.average = [], [], []

        self.Save_Path = 'Models'
        self.state_size = (self.REM_STEP, self.ROWS, self.COLS)
        
        if not os.path.exists(self.Save_Path): os.makedirs(self.Save_Path)
        self.path = '{}_A3C_{}'.format(self.env_name, self.learning_rate)
        self.model_name = os.path.join(self.Save_Path, self.path)

        # Create Actor-Critic network model
        self.policy = ActorCritic(input_shape=self.state_size, action_space=self.action_size)
        self.policy_old = ActorCritic(input_shape=self.state_size, action_space=self.action_size)
        
        self.optimizer = tf.keras.optimizers.Adam(self.learning_rate)

    @tf.function
    def act_old(self, state, memory_state, carry_state, training):
        # Use the network to predict the next action to take, using the model
        prediction = self.policy_old(tf.expand_dims(state, 0), memory_state, carry_state, training=training)
        action = tf.random.categorical(prediction[0], 1)
        
        return action[0][0], prediction
    
    @tf.function
    def act_new(self, state, memory_state, carry_state, training):
        # Use the network to predict the next action to take, using the model
        prediction = self.policy(tf.expand_dims(state, 0), memory_state, carry_state, training=training)
        action = tf.random.categorical(prediction[0], 1)
        
        return action[0][0], prediction

    def discount_rewards(self, reward):
        # Compute the gamma-discounted rewards over an episode
        gamma = 0.99    # discount rate
        running_add = 0
        discounted_r = np.zeros_like(reward)
        for i in reversed(range(0, len(reward))):
            if reward[i] != 0: # reset the sum, since this was a game boundary (pong specific!)
                running_add = 0

            running_add = running_add * gamma + reward[i]
            discounted_r[i] = running_add
    
        if np.std(discounted_r) != 0:
            discounted_r -= np.mean(discounted_r) # normalizing the result
            discounted_r /= np.std(discounted_r) # divide by standard deviation

        return discounted_r
    
    def get_loss(self, states, actions, discounted_r, memory_state_old, carry_state_old, memory_state_new, carry_state_new):
        batch_size = states.shape[0]
        
        action_logits_old = tf.TensorArray(dtype=tf.float32, size=0, dynamic_size=True)
        values_old = tf.TensorArray(dtype=tf.float32, size=0, dynamic_size=True)
        
        action_logits = tf.TensorArray(dtype=tf.float32, size=0, dynamic_size=True)
        values = tf.TensorArray(dtype=tf.float32, size=0, dynamic_size=True)
        for i in range(0, batch_size):
            action_old, prediction_old = self.act_old(states[i,:,:,:], memory_state_old, carry_state_old, training=True)
            action_logit_old = prediction_old[0]
            value_old = prediction_old[1]
            memory_state_old = prediction_old[2]
            carry_state_old = prediction_old[3]

            action_logits_old = action_logits_old.write(i, action_logit_old[0])
            values_old = values_old.write(i, tf.squeeze(value_old))
            
            action_new, prediction_new = self.act_new(states[i,:,:,:], memory_state_new, carry_state_new, training=True)
            action_logit = prediction_new[0]
            value = prediction_new[1]
            memory_state_new = prediction_new[2]
            carry_state_new = prediction_new[3]
            
            action_logits = action_logits.write(i, action_logit[0])
            values = values.write(i,  tf.squeeze(value))
            
        action_logits_old = action_logits_old.stack()
        values_old = values_old.stack()
        action_logits = action_logits.stack()
        values = values.stack()
        
        action_logits_selected_old = take_vector_elements(action_logits_old, actions)
        
        action_logits_selected = take_vector_elements(action_logits, actions)
        
        advantages = discounted_r - tf.stop_gradient(values_old)
            
        entropy_loss_old = tf.keras.losses.categorical_crossentropy(action_logits_old, action_logits_old)
            
        action_logits_selected_old = tf.nn.softmax(action_logits_selected_old)
        action_log_selected_old = tf.math.log(action_logits_selected_old)
        
        action_logits_selected = tf.nn.softmax(action_logits_selected)
        action_log_selected = tf.math.log(action_logits_selected)
            
        ratios = tf.math.exp(action_log_selected - tf.stop_gradient(action_log_selected_old))
            
        eps_clip = 0.2 
        surr1 = ratios * advantages
        surr2 = tf.clip_by_value(ratios, 1 - eps_clip, 1 + eps_clip) * advantages
            
        actor_loss = -tf.math.minimum(surr1, surr2)
        actor_loss = tf.math.reduce_mean(actor_loss)
        actor_loss = tf.cast(actor_loss, 'float32')
        #print("actor_loss: ", actor_loss)
            
        critic_loss = mse_loss(values, discounted_r)
        total_loss = actor_loss + critic_loss
        
        return total_loss, memory_state_old, carry_state_old, memory_state_new, carry_state_new
        
    def replay(self, states, actions, rewards, memory_state, carry_state):
        # reshape memory to appropriate shape for training
        states = np.vstack(states)
        #print("states.shape: ", states.shape)
        
        # Compute discounted rewards
        discounted_r = self.discount_rewards(rewards)
        discounted_r = discounted_r.astype(np.float32)
        #print("discounted_r: ", discounted_r)
        
        divide_size = 500
        batch_size = states.shape[0]
        #print("batch_size: ", batch_size)
        epoch_size = batch_size // divide_size
        #print("epoch_size: ", epoch_size)
        remain_size = batch_size - epoch_size * divide_size
        #print("remain_size: ", remain_size)
        
        memory_state_old = memory_state
        carry_state_old = carry_state
        memory_state_new = memory_state
        carry_state_new = carry_state
        for e in range(0, epoch_size):
            with tf.GradientTape() as tape:
                return_value = self.get_loss(states[divide_size*e:divide_size*(e+1),:,:,:], 
                                             actions[divide_size*e:divide_size*(e+1)], 
                                             discounted_r[divide_size*e:divide_size*(e+1)],
                                             memory_state_old, carry_state_old, memory_state_new, carry_state_new)
                
            total_loss = return_value[0]
            memory_state_old = return_value[1]
            carry_state_old = return_value[2]
            memory_state_new = return_value[3]
            carry_state_new = return_value[4]
            #print("total_loss: ", total_loss)
            #print("")
            grads = tape.gradient(total_loss, self.policy.trainable_variables)
            self.optimizer.apply_gradients(zip(grads, self.policy.trainable_variables))
            
        if remain_size != 0:
            with tf.GradientTape() as tape:
                total_loss = self.get_loss(states[divide_size*epoch_size:divide_size*epoch_size+remain_size,:,:,:], 
                                           actions[divide_size*epoch_size:divide_size*epoch_size+remain_size], 
                                           discounted_r[divide_size*epoch_size:divide_size*epoch_size+remain_size],
                                           memory_state_old, carry_state_old, memory_state_new, carry_state_new)
 
        for a, b in zip(self.policy_old.variables, self.policy.variables):
            a.assign(b)  # copies the variables of model_b into model_a
        
        return total_loss
        
    def load(self, model_name):
        self.ActorCritic = load_model(model_name, compile=False)
        #self.Critic = load_model(Critic_name, compile=False)

    def save(self):
        self.ActorCritic.save(self.model_name)
        #self.Critic.save(self.Model_name + '_Critic.h5')

    def GetScoreAverage(self, score, episode):
        self.scores.append(score)
        self.episodes.append(episode)
        self.average.append(sum(self.scores[-50:]) / len(self.scores[-50:]))
        
        return self.average[-1]
    
    def imshow(self, image, rem_step=0):
        cv2.imshow(self.model_name + str(rem_step), image[rem_step,...])
        if cv2.waitKey(25) & 0xFF == ord("q"):
            cv2.destroyAllWindows()
            return

    def reset(self, env):
        image_memory = np.zeros(self.state_size)
        env.reset()
        state = env.observations()
        state = state['RGB_INTERLEAVED']
            
        return state
    
    def step(self, action, env, image_memory):
        reward = env.step(ACTIONS[ACTION_LIST[action]])
        if not env.is_running():
            done = True
            next_state = np.zeros((64,64,3))
            reward = 0
        else:
            done = False
            next_state = env.observations()
            next_state = next_state['RGB_INTERLEAVED']
        
        return next_state, reward, done
    
    def train(self, n_threads):
        self.env.close()
        # Instantiate one environment per thread
        envs = [deepmind_lab.Lab(self.env_name, ['RGB_INTERLEAVED'], 
                                 {'fps': '15', 'width': '64', 'height': '64'}) for i in range(n_threads)]
        # Create threads
        threads = [threading.Thread(
                target=self.train_threading,
                daemon=True,
                args=(self,
                    envs[i],
                    i)) for i in range(n_threads)]

        for t in threads:
            time.sleep(2)
            t.start()
            
        for t in threads:
            time.sleep(10)
            t.join()
            
    def render(self, obs):
        im_rgb = cv2.cvtColor(obs, cv2.COLOR_BGR2RGB)
        cv2.imshow("obs", im_rgb)
        cv2.waitKey(1)
            
    def train_threading(self, agent, env, thread):
        total_step = 0
        while self.episode < self.EPISODES:
            # Reset episode
            score, done, SAVING = 0, False, ''
            state = self.reset(env)
            state = state / 255.0
            state = state.astype(np.float32)

            states, actions, rewards = [], [], []
            memory_state = np.zeros([1,64], dtype=np.float32)
            carry_state = np.zeros([1,64], dtype=np.float32)
            
            initial_memory_state = memory_state
            initial_carry_state = carry_state
            while True:
                action, prediction = agent.act_old(state, memory_state, carry_state, training=False)
                #print("action: ", action)
                memory_state = prediction[2]
                carry_state = prediction[3]
                #print("action: ", action)
                next_state, reward, done = self.step(action.numpy(), env, state)
                #print("next_state: ", next_state)
                next_state = next_state / 255.0
                next_state = next_state.astype(np.float32)
                #print("next_state.shape: ", next_state.shape)
                
                if thread == 0:
                    self.render(next_state)
                
                if done == True:
                    break
                    
                states.append(np.array([state]))
                actions.append(action)
                rewards.append(reward)

                score += reward
                state = next_state
            
            self.lock.acquire()
            self.replay(states, actions, rewards, initial_memory_state, initial_carry_state)
            states, actions, rewards = [], [], []
            self.lock.release()
                
            # Update episode count
            with self.lock:
                average = self.GetScoreAverage(score, self.episode)
                with writer.as_default():
                    # other model code would go here
                    tf.summary.scalar("average", average, step=self.episode)
                    writer.flush()
                
                # saving best models
                if average >= self.max_average:
                    self.max_average = average
                    #self.save()
                    SAVING = "SAVING"
                else:
                    SAVING = ""

                print("episode: {}/{}, thread: {}, score: {}, average: {:.2f} {}".format(self.episode, self.EPISODES, thread, score, average, SAVING))
                if(self.episode < self.EPISODES):
                    self.episode += 1

        env.close()            

    def test(self, Actor_name, Critic_name):
        self.load(Actor_name, Critic_name)
        for e in range(100):
            state = self.reset(self.env)
            done = False
            score = 0
            while not done:
                self.env.render()
                action = np.argmax(self.Actor.predict(state))
                state, reward, done, _ = self.step(action, self.env, state)
                score += reward
                if done:
                    print("episode: {}/{}, score: {}".format(e, self.EPISODES, score))
                    break

        self.env.close()


if __name__ == "__main__":
    env_name = 'ctf_simple'
    agent = PPOAgent(env_name)
    
    #agent.run() # use as A2C
    agent.train(n_threads=5) # use as A3C
    #agent.test('Models/Pong-v0_A3C_2.5e-05_Actor.h5', '')